## 🔐 Prerequisites

Before running the first cell, make sure you're authenticated with Azure CLI:

```bash
az login
```

**Redis Server Required:**
```bash
# Using Docker:
docker run -d -p 6379:6379 redis:latest
```

# 🏦 Redis Chat Message Store for Threads

## Industry Use Case: Distributed Customer Session Management

This notebook demonstrates how to use **Redis** for managing conversation threads across distributed banking applications.

| Feature | FSI Application |
|---------|----------------|
| **Distributed Sessions** | Scale across multiple app instances |
| **Session Persistence** | Customer conversations survive restarts |
| **Message Limits** | Automatic conversation trimming |
| **User Isolation** | Separate threads per customer/session |

In [ ]:
import os
from uuid import uuid4
from dotenv import load_dotenv
from azure.identity import AzureCliCredential
from agent_framework import AgentThread
from agent_framework.azure import AzureOpenAIChatClient
from agent_framework.redis import RedisChatMessageStore

load_dotenv('../../.env', override=True)

print(f"✅ Environment loaded")
print(f"Azure OpenAI Endpoint: {os.getenv('AZURE_OPENAI_ENDPOINT')}")

In [ ]:
async def get_account_balance(account_id: str) -> str:
    """Get account balance for a customer."""
    accounts = {
        "ACC-1001": {"balance": 25430.50, "type": "Checking", "currency": "USD"},
        "ACC-1002": {"balance": 150000.00, "type": "Savings", "currency": "USD"},
    }
    a = accounts.get(account_id, {"balance": 0, "type": "Unknown", "currency": "USD"})
    return f"Account {account_id}: {a['type']} - Balance ${a['balance']:,.2f} {a['currency']}"


async def transfer_funds(from_account: str, to_account: str, amount: float) -> str:
    """Transfer funds between accounts (simulated)."""
    return f"Transfer initiated: ${amount:,.2f} from {from_account} to {to_account}. Reference: TXN-{uuid4().hex[:8].upper()}"


print("✅ Tools defined: get_account_balance, transfer_funds")

## Example 1: Basic Redis Store

Create a Redis-backed conversation with automatic persistence.

In [ ]:
async def example_basic_redis_store():
    """Basic Redis chat message store example."""
    print("\n--- Basic Redis Chat Message Store ---\n")
    
    # Create Redis store with auto-generated thread ID
    redis_store = RedisChatMessageStore(redis_url="redis://localhost:6379")
    print(f"✅ Created store with thread ID: {redis_store.thread_id}")
    
    # Create thread with Redis store
    thread = AgentThread(message_store=redis_store)
    
    # Create agent using AzureOpenAIChatClient (supports custom message stores)
    # Note: AzureAIAgentClient uses service-managed threads which conflict with custom stores
    endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME", "gpt-4o"))
    
    agent = AzureOpenAIChatClient(
        endpoint=endpoint,
        deployment_name=deployment,
        credential=AzureCliCredential(),
    ).as_agent(
        name="BankingSupport",
        instructions="You are a helpful banking support agent. Help customers with account inquiries.",
        tools=[get_account_balance, transfer_funds],
    )
    
    # Have a conversation
    query1 = "Hello! Can you check my balance for account ACC-1001?"
    print(f"Customer: {query1}")
    response1 = await agent.run(query1, thread=thread)
    print(f"Agent: {response1.text}\n")
    
    query2 = "Thanks! What about my savings account ACC-1002?"
    print(f"Customer: {query2}")
    response2 = await agent.run(query2, thread=thread)
    print(f"Agent: {response2.text}\n")
    
    # Check Redis storage
    messages = await redis_store.list_messages()
    print(f"📊 Messages in Redis: {len(messages)}")
    
    # Cleanup
    await redis_store.clear()
    await redis_store.aclose()
    print("✅ Redis data cleaned up\n")

await example_basic_redis_store()

In [ ]:
## Example 2: User Session Management

Track conversations per customer with isolated session storage.

async def example_user_session_management():
    """User session management with Redis."""
    print("\n--- User Session Management ---\n")
    
    customer_id = "CUST-12345"
    session_id = f"session_{uuid4().hex[:8]}"
    
    # Create Redis store for specific customer session
    def create_customer_session_store():
        return RedisChatMessageStore(
            redis_url="redis://localhost:6379",
            thread_id=f"customer_{customer_id}_{session_id}",
            max_messages=10,  # Keep only last 10 messages
        )
    
    # Create agent with factory pattern using AzureOpenAIChatClient
    endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME", "gpt-4o"))
    
    agent = AzureOpenAIChatClient(
        endpoint=endpoint,
        deployment_name=deployment,
        credential=AzureCliCredential(),
    ).as_agent(
        name="BankingSupport",
        instructions="You are a banking support agent. Remember customer preferences.",
        tools=[get_account_balance, transfer_funds],
        chat_message_store_factory=create_customer_session_store,
    )
    
    thread = agent.get_new_thread()
    print(f"✅ Session started for customer {customer_id}")
    if hasattr(thread.message_store, "thread_id"):
        print(f"   Thread ID: {thread.message_store.thread_id}\n")
    
    # Simulate conversation
    queries = [
        "Hi, I prefer communication in a formal tone.",
        "Check my balance for ACC-1001 please.",
        "Can you remember my preference for future conversations?",
    ]
    
    for i, query in enumerate(queries, 1):
        print(f"Customer [{i}]: {query}")
        response = await agent.run(query, thread=thread)
        print(f"Agent: {response.text}\n")
    
    # Check stored messages
    if thread.message_store:
        messages = await thread.message_store.list_messages()
        print(f"📊 Messages stored for {customer_id}: {len(messages)}")
        await thread.message_store.clear()
        await thread.message_store.aclose()
    
    print("✅ Session data cleaned up\n")

await example_user_session_management()

In [ ]:
## Example 3: Conversation Persistence Across Restarts

Simulate resuming a conversation after application restart.

In [ ]:
async def example_conversation_persistence():
    """Conversation persistence across app restarts."""
    print("\n--- Conversation Persistence Across Restarts ---\n")
    
    conversation_id = "persistent_banking_session"
    
    # Create agent using AzureOpenAIChatClient
    endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME", "gpt-4o"))
    
    agent = AzureOpenAIChatClient(
        endpoint=endpoint,
        deployment_name=deployment,
        credential=AzureCliCredential(),
    ).as_agent(
        name="BankingSupport",
        instructions="You are a banking support agent.",
        tools=[get_account_balance, transfer_funds],
    )
    
    # Phase 1: Start conversation
    print("=== Phase 1: Starting conversation ===")
    store1 = RedisChatMessageStore(
        redis_url="redis://localhost:6379",
        thread_id=conversation_id,
    )
    thread1 = AgentThread(message_store=store1)
    
    query1 = "I need to transfer $500 from ACC-1001 to ACC-1002"
    print(f"Customer: {query1}")
    response1 = await agent.run(query1, thread=thread1)
    print(f"Agent: {response1.text}")
    print(f"\n📊 Stored {len(await store1.list_messages())} messages in Redis")
    await store1.aclose()
    
    # Phase 2: Resume after "restart"
    print("\n=== Phase 2: Resuming after 'app restart' ===")
    store2 = RedisChatMessageStore(
        redis_url="redis://localhost:6379",
        thread_id=conversation_id,  # Same thread ID
    )
    thread2 = AgentThread(message_store=store2)
    
    query2 = "What transfer did I just request?"
    print(f"Customer: {query2}")
    response2 = await agent.run(query2, thread=thread2)
    print(f"Agent: {response2.text}")
    
    print(f"\n📊 Total messages after resume: {len(await store2.list_messages())}")
    
    # Cleanup
    await store2.clear()
    await store2.aclose()
    
    print("✅ Persistence demo completed\n")

await example_conversation_persistence()

## Key Takeaways

| Feature | Description |
|---------|-------------|
| `RedisChatMessageStore` | Built-in Redis storage for conversations |
| `thread_id` | Unique identifier for conversation isolation |
| `max_messages` | Automatic trimming for memory management |
| `chat_message_store_factory` | Factory pattern for per-thread stores |

## Redis Benefits for FSI

| Benefit | Application |
|---------|-------------|
| **High Performance** | Low-latency customer interactions |
| **Distributed** | Share sessions across app instances |
| **Persistence** | Survive application restarts |
| **Scalability** | Handle high customer volume |